In [2]:
pip install tenacity

In [1]:
pip install umls-api

  Created wheel for umls-api: filename=umls_api-0.1.0-py3-none-any.whl size=3408 sha256=08f7bd461b34a3a4b14842d03a2b28ef0db2f9063ba48eaf7ec64dfef3eaa0ec
  Stored in directory: /root/.cache/pip/wheels/b1/0c/16/429473df1665cab9602747448f46d5fb701bf2e9634915ea71
Successfully built umls-api


In [3]:
pip install stanza

     |████████████████████████████████| 342 kB 7.7 MB/s 


In [4]:
import stanza
stanza.download('en',package = 'craft')

2021-08-18 10:34:56 INFO: Downloading these customized packages for language: en (English)...
| Processor | Package |
-----------------------
| tokenize  | craft   |
| pos       | craft   |
| lemma     | craft   |
| depparse  | craft   |
| pretrain  | craft   |



2021-08-18 10:35:28 INFO: Finished downloading models and saved to /root/stanza_resources.


In [5]:
# import requests
# import json
# from lxml.html import fromstring
# import pyquery
import pandas as pd
# import re
import numpy as np
# import umls_api
import gensim

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
from gensim.parsing.preprocessing import strip_numeric,strip_multiple_whitespaces, strip_punctuation, strip_short, strip_tags

In [7]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('drive/MyDrive/bbc_sent_final.csv',encoding = 'utf-8')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,text,sent
0,Psychiatrists are being urged to ask children ...,0
1,The number of adults seeking help to cope with...,-1
2,The current system of checking newborns for hi...,-1
3,Calorie-filled Easter eggs are being sold in s...,-1
4,"""She is so desperate to end it all, she curren...",-1


In [8]:
new_sw = ['people','say','says','said','could','one','would','take','want','have','she','he','give','told','know','but','health','month','day','year', 'minute','second','hundred','thousand','however']
stop_words = STOPWORDS.union(set(new_sw))

In [ ]:
# pubmed_stoplist = ['a','about','again','all','almost', 'also','although','always',\
#                    'among','an','and','another','any','are','as','at','be','because',\
#                    'been','before','being','between','both','but','by','can','could',\
#                    'did','do','does','done','due','during','each','either','enough', \
#                    'especially','etc','for','found', 'from','further','had','is','it',\
#                    'has','have','having','here','how','however','i','if','in','into',\
#                    'its','itself','just','kg','km', 'made','mainly','make','may','mg',\
#                    'might','ml','mm','most','mostly','must','nearly','neither','no',\
#                    'nor','obtained','of','often','on','our','overall','perhaps','pmid',\
#                    'quite','rather','really','regarding','seem','seen','several',\
#                    'should','show','showed','shown','shows','significantly','since',\
#                    'so','some','such','than','that','the','their','theirs','them',\
#                    'then','there','therefore','these','they','this','those','through',\
#                    'thus','to','upon','use','used','using','various','very', 'was','we',\
#                    'were','what','when', 'which','while','with','within','without','would']

# stop_words = STOPWORDS.union(set(pubmed_stoplist))

In [21]:
def cleaning(text):
  # clean =[]
  text = strip_multiple_whitespaces(text)
  # text = strip_numeric(text)
  text = strip_punctuation(text)
  text = strip_tags(text)
  # text = strip_short(text,minsize=3)
  text = remove_stopwords(text)
  text = text.lower()
  # # lower case, de-accents.  Convert the texts into a list of tokens
  # for token in gensim.utils.simple_preprocess(text, deacc=True):
  #   clean.append(token)
  lem_text = nlp_craft(text)
  return lem_text

In [14]:
nlp_craft = stanza.Pipeline('en',package = 'craft',processor = 'pos,tokenize,lemma',tokenize_pretokenized = True)

2021-08-18 11:06:40 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | craft   |
| pos       | craft   |
| lemma     | craft   |
| depparse  | craft   |

2021-08-18 11:06:40 INFO: Use device: gpu
2021-08-18 11:06:40 INFO: Loading: tokenize
2021-08-18 11:06:40 INFO: Loading: pos
2021-08-18 11:06:47 INFO: Loading: lemma
2021-08-18 11:06:47 INFO: Loading: depparse
2021-08-18 11:06:47 INFO: Done loading processors!


In [22]:
df['clean_text'] = df.text.apply(cleaning)

In [23]:
df.drop('lem_text', axis=1, inplace=True)

In [18]:
def token_list(nlp_doc):
  token_list = []
  for i in range(len(nlp_doc.sentences)):
    article_token_list = []
    for m in range(len(nlp_doc.sentences[i].tokens)):
      token = nlp_doc.sentences[i].words[m].lemma
      if token not in stop_words or len(token)>2:
        article_token_list.append(token)
    token_list.append(article_token_list)
  return token_list

In [24]:
df['token_list'] = df.clean_text.apply(token_list)

In [25]:
df.head()

,text,sent,clean_text,token_list
0,Psychiatrists are being urged to ask children ...,0,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[psychiatrist, urge, ask, child, mental, heal..."
1,The number of adults seeking help to cope with...,-1,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[the, number, adult, seek, help, cope, alcoho..."
2,The current system of checking newborns for hi...,-1,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[the, current, check, newborn, hip, problem, ..."
3,Calorie-filled Easter eggs are being sold in s...,-1,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[calorie, fill, easter, egg, sell, shop, far,..."
4,"""She is so desperate to end it all, she curren...",-1,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[she, desperate, end, currently, toothbrush, ..."


In [30]:
type_dict = pd.read_csv('drive/MyDrive/bbc_type_dict_final.csv',encoding = 'utf-8')

In [33]:
umls_type = {
    'Acquired Abnormality': -1,
    'Anatomical Abnormality': -1,
    'Antibiotic': 1,
    'Biological Active Substance': 1,
    'Biomedical or Dental Material': 1,
    'Cell or Molecular Dysfunction': -1,
    'Clinical Drug': 1,
    'Congenital Abnormality': -1,
    'Disease and Syndrome': -1,
    'Drug Delivery Device': 1,
    'Hazardous or Poisonous Substance': -1,
    'Health Care Activity': 1,
    'Injury or Poisoning': -1,
    'Medical Device': 1,
    'Mental or Behavioural Dysfunction': -1,
    'Neoplastic Process': -1,
    'Pathological Function': -1,
    'Pharmacologic Substance':1,
    'Sign or Symptom':-1,
    'Therapeutic or Preventive Procedure': 1
}

In [69]:
def change_type(token_list):
  new_token_list = []
  for l in token_list:
    for token in l:
      try: 
        if type_dict[token] in umls_type.keys():
          new_token_list.append(type_dict[token])
        else:
          new_token_list.append(token)
      except:
         new_token_list.append(token)
  return new_token_list

In [70]:
df['umls_token']=df.token_list.apply(change_type)

In [71]:
df.head()

,text,sent,clean_text,token_list,umls_token
0,Psychiatrists are being urged to ask children ...,0,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[psychiatrist, urge, ask, child, mental, heal...","[psychiatrist, urge, ask, child, mental, healt..."
1,The number of adults seeking help to cope with...,-1,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[the, number, adult, seek, help, cope, alcoho...","[the, number, adult, seek, help, cope, alcohol..."
2,The current system of checking newborns for hi...,-1,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[the, current, check, newborn, hip, problem, ...","[the, current, check, newborn, hip, problem, f..."
3,Calorie-filled Easter eggs are being sold in s...,-1,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[calorie, fill, easter, egg, sell, shop, far,...","[calorie, fill, easter, egg, sell, shop, far, ..."
4,"""She is so desperate to end it all, she curren...",-1,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[she, desperate, end, currently, toothbrush, ...","[she, desperate, end, currently, toothbrush, i..."


In [74]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [76]:
sia = SIA()
sia.lexicon.update(umls_type)

In [77]:
def join_text(text_list):
  return' '.join(text_list)

In [78]:
df['joint_text'] = df.umls_token.apply(join_text)

In [79]:
def polarity(text):
  pol_score = sia.polarity_scores(text)
  return pol_score

In [80]:
df['polarity_score'] = df.joint_text.apply(polarity)

In [81]:
def label(polarity_score):
  if polarity_score['compound'] > 0.2 :
    return 1
  elif polarity_score['compound'] < -0.2 :
    return -1
  else:
    return 0

In [82]:
 df['label']=df.polarity_score.apply(label)

In [83]:
df.head()

,text,sent,clean_text,token_list,umls_token,joint_text,polarity_score,label
0,Psychiatrists are being urged to ask children ...,0,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[psychiatrist, urge, ask, child, mental, heal...","[psychiatrist, urge, ask, child, mental, healt...",psychiatrist urge ask child mental health issu...,"{'neg': 0.136, 'neu': 0.776, 'pos': 0.088, 'co...",-1
1,The number of adults seeking help to cope with...,-1,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[the, number, adult, seek, help, cope, alcoho...","[the, number, adult, seek, help, cope, alcohol...",the number adult seek help cope alcoholic pare...,"{'neg': 0.191, 'neu': 0.648, 'pos': 0.161, 'co...",-1
2,The current system of checking newborns for hi...,-1,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[the, current, check, newborn, hip, problem, ...","[the, current, check, newborn, hip, problem, f...",the current check newborn hip problem fail res...,"{'neg': 0.164, 'neu': 0.789, 'pos': 0.047, 'co...",-1
3,Calorie-filled Easter eggs are being sold in s...,-1,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[calorie, fill, easter, egg, sell, shop, far,...","[calorie, fill, easter, egg, sell, shop, far, ...",calorie fill easter egg sell shop far early te...,"{'neg': 0.075, 'neu': 0.795, 'pos': 0.13, 'com...",1
4,"""She is so desperate to end it all, she curren...",-1,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","[[she, desperate, end, currently, toothbrush, ...","[she, desperate, end, currently, toothbrush, i...",she desperate end currently toothbrush inside ...,"{'neg': 0.159, 'neu': 0.634, 'pos': 0.206, 'co...",1


In [86]:
comparison = np.where(df.sent == df.label, True, False)

In [87]:
np.count_nonzero(comparison)

45